In [1]:
import wbgapi as wb
import pandas as pd

indicators = {
    "School enrollment, primary (% gross)": "SE.PRM.ENRR",
    "School enrollment, secondary (% gross)": "SE.SEC.ENRR",
    "School enrollment, tertiary (% gross)": "SE.TER.ENRR",
}

# Define the countries
countries = ["JPN", "AFG", "IND"]  # ISO3 country codes

def create_enrollment_table(country_code, country_name, indicators_dict):
    """
    Fetches and formats school enrollment data for a specific country.

    Args:
        country_code (str): ISO3 code of the country.
        country_name (str): Name of the country.
        indicators_dict (dict): Dictionary of indicator names and codes.

    Returns:
        pandas.DataFrame: A table with years as index and enrollment indicators as columns.
                          Returns None if no data is found.
    """
    data = wb.data.DataFrame(indicators_dict.values(), country_code, mrv=200) # Fetch last 200 years of data
    if data.empty:
        print(f"No data found for {country_name}.")
        return None

    df = data.transpose()
    df.index.name = "Year"
    df.columns = list(indicators_dict.keys())
    df["Country"] = country_name
    df = df.set_index("Country", append=True)
    return df

# Create tables for each country
japan_table = create_enrollment_table("JPN", "Japan", indicators)
afghanistan_table = create_enrollment_table("AFG", "Afghanistan", indicators)
india_table = create_enrollment_table("IND", "India", indicators)

In [2]:
import sqlite3

conn = sqlite3.connect(':memory:')

# Example: Save the India table properly to SQLite
india_table_reset = india_table.reset_index()
india_table_reset.to_sql("india", conn, if_exists="replace", index=False)
japan_table_reset = japan_table.reset_index()
japan_table_reset.to_sql("japan", conn, if_exists="replace", index=False)
afghanistan_table_reset = afghanistan_table.reset_index()
afghanistan_table_reset.to_sql("afghanistan", conn, if_exists="replace", index=False)


47

In [3]:
conn.execute('ALTER TABLE india RENAME COLUMN "School enrollment, primary (% gross)" TO "Primary Enrollment";')
conn.execute('ALTER TABLE india RENAME COLUMN "School enrollment, secondary (% gross)" TO "Secondary Enrollment";')
conn.execute('ALTER TABLE india RENAME COLUMN "School enrollment, tertiary (% gross)" TO "Tertiary Enrollment";')
conn.execute('ALTER TABLE japan RENAME COLUMN "School enrollment, primary (% gross)" TO "Primary Enrollment";')
conn.execute('ALTER TABLE japan RENAME COLUMN "School enrollment, secondary (% gross)" TO "Secondary Enrollment";')
conn.execute('ALTER TABLE japan RENAME COLUMN "School enrollment, tertiary (% gross)" TO "Tertiary Enrollment";')
conn.execute('ALTER TABLE afghanistan RENAME COLUMN "School enrollment, primary (% gross)" TO "Primary Enrollment";')
conn.execute('ALTER TABLE afghanistan RENAME COLUMN "School enrollment, secondary (% gross)" TO "Secondary Enrollment";')
conn.execute('ALTER TABLE afghanistan RENAME COLUMN "School enrollment, tertiary (% gross)" TO "Tertiary Enrollment";')

In [5]:
pd.read_sql_query("SELECT * FROM india", conn).to_csv("india_enrollment.csv", index=False)
pd.read_sql_query("SELECT * FROM japan", conn).to_csv("japan_enrollment.csv", index=False)
pd.read_sql_query("SELECT * FROM afghanistan", conn).to_csv("afghanistan_enrollment.csv", index=False)

**Descriptive Statistics**

In [4]:
def get_descriptive_stats(table_name):
    query = f"""
    SELECT 
      COUNT(*) AS count,
      AVG("Primary Enrollment") AS avg_primary,
      MIN("Primary Enrollment") AS min_primary,
      MAX("Primary Enrollment") AS max_primary,
      AVG("Secondary Enrollment") AS avg_secondary,
      MIN("Secondary Enrollment") AS min_secondary,
      MAX("Secondary Enrollment") AS max_secondary,
      AVG("Tertiary Enrollment") AS avg_tertiary,
      MIN("Tertiary Enrollment") AS min_tertiary,
      MAX("Tertiary Enrollment") AS max_tertiary
    FROM {table_name};
    """
    return pd.read_sql_query(query, conn)

# Descriptive statistics for each table
india_stats = get_descriptive_stats("india")
japan_stats = get_descriptive_stats("japan")
afghanistan_stats = get_descriptive_stats("afghanistan")

# Print the results
print("India:\n", india_stats)
print("\nJapan:\n", japan_stats)
print("\nAfghanistan:\n", afghanistan_stats)

India:
    count  avg_primary  min_primary  max_primary  avg_secondary  min_secondary  \
0     53    96.764678    78.981659   119.512833      51.181328       24.17012   

   max_secondary  avg_tertiary  min_tertiary  max_tertiary  
0      81.177861     13.906582       4.92445     33.116524  

Japan:
    count  avg_primary  min_primary  max_primary  avg_secondary  min_secondary  \
0     39   102.726731    98.300819   106.348808      97.819013      89.496887   

   max_secondary  avg_tertiary  min_tertiary  max_tertiary  
0     105.892799     38.852622      16.85821      64.62883  

Afghanistan:
    count  avg_primary  min_primary  max_primary  avg_secondary  min_secondary  \
0     47    62.809384     21.81455   110.007507      26.310449        8.88566   

   max_secondary  avg_tertiary  min_tertiary  max_tertiary  
0      57.039299      2.935639       0.80651      10.85844  
